# The sensitivity analysis in pyPowsybl 

This notebook illustrates how to run a sensitivity analysis.


In [1]:
pip install pypowsybl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://devin-depot.rte-france.com/repository/pypi-all/simple
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to

In [2]:
import pypowsybl as pp

In [3]:
n6 = pp.network.create_metrix_tutorial_six_buses_network()

## 1 - DC Sensitivity analysis

To perform a sensitivity analysis, you must first define the “factors” you want to compute. What we call a factor is the impact of a small variation of a variable, typically the active power injection of a generator, a load or a phase shifter, on a function, typically the active power flow on a branch. 

To make the definition of those factors easier, `pypowsybl` provides a method to define the variables (injection, phase shifter) through their ids (variables_ids) and the functions through the branch ids (branches_ids). We obtain a matrix of sensitivities as a result:

In [4]:
analysis = pp.sensitivity.create_dc_analysis()
analysis.add_branch_flow_factor_matrix(branches_ids=['S_SO_1', 'S_SO_2'], variables_ids=['SE_G'])
result = analysis.run(n6)
result.get_reference_matrix()

,S_SO_1,S_SO_2
reference_values,-120.433341,-120.433341


In [5]:
result.get_sensitivity_matrix()

,S_SO_1,S_SO_2
SE_G,0.26087,0.26087


The result can be interpreted in the following way: an increase of 1 MW on generator SE_G impacts the lines S_SO_1 and S_SO_2 with a 0.3 MW increase of the active power flow from side 2 to side 1.

## 2 - Zone to zone sensitivity: PTDF computation

This zone to zone sensitivity feature is better known as Power Transfer Distribution Factor (PTDF).

Consider now that the node `SE` is now in Italy:

In [6]:
n6b = pp.network.create_metrix_tutorial_six_buses_network()
n6b.update_substations(id=['SE'], TSO=['Terna'], country=['IT'])
n6b.update_substations(id=['NO', 'S', 'SO', 'N'], TSO=['RTE', 'RTE', 'RTE', 'RTE'], country=['FR', 'FR', 'FR', 'FR'])

In [7]:
pp.loadflow.run_dc(n6b)

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=0, reference_bus_id='NE_poste_0', slack_bus_results=[SlackBusResult(id='NE_poste_0', active_power_mismatch=4.440892098500626e-14)], distributed_active_power=nan)]

In [8]:
n6b.get_substations()

,name,TSO,geo_tags,country
id,,,,
NO,,RTE,,FR
S,,RTE,,FR
SO,,RTE,,FR
SE,,Terna,,IT
N,,RTE,,FR


In [9]:
n6b.get_generators()

,name,energy_source,target_p,min_p,max_p,min_q,max_q,rated_s,reactive_limits_kind,target_v,target_q,voltage_regulator_on,regulated_element_id,p,q,i,voltage_level_id,bus_id,connected
id,,,,,,,,,,,,,,,,,,,
SO_G1,,THERMAL,480.0,0.0,2000.0,NaN,NaN,NaN,CURVE,406.450043,2.35231,True,SO_G1,-436.521739,-0.0,NaN,SO_poste,SO_poste_0,True
SO_G2,,THERMAL,480.0,0.0,2000.0,NaN,NaN,NaN,CURVE,406.450043,2.35231,True,SO_G2,-436.521739,-0.0,NaN,SO_poste,SO_poste_0,True
SE_G,,THERMAL,100.0,0.0,600.0,NaN,NaN,NaN,CURVE,406.450043,6.45498,True,SE_G,-86.956522,-0.0,NaN,SE_poste,SE_poste_0,True
N_G,,THERMAL,0.0,0.0,600.0,NaN,NaN,NaN,CURVE,406.450043,7.48158,True,N_G,-0.000000,-0.0,NaN,N_poste,N_poste_0,True


In [10]:
n6b.get_loads()

,name,type,p0,q0,p,q,i,voltage_level_id,bus_id,connected
id,,,,,,,,,,
SO_L,,UNDEFINED,480.0,4.8,480.0,4.8,NaN,SO_poste,SO_poste_0,True
SE_L1,,UNDEFINED,480.0,4.8,480.0,4.8,NaN,SE_poste,SE_poste_0,True
SE_L2,,UNDEFINED,0.0,4.8,0.0,4.8,NaN,SE_poste,SE_poste_0,True


In [11]:
zone_fr = pp.sensitivity.create_country_zone(n6b, 'FR')
zone_it = pp.sensitivity.create_country_zone(n6b, 'IT')

<enum 'ZoneKeyType'>
<enum 'ZoneKeyType'>
True
<enum 'ZoneKeyType'>
<enum 'ZoneKeyType'>
True


In [12]:
zone_fr.shift_keys_by_injections_ids

{'SO_G1': 480.0, 'SO_G2': 480.0, 'N_G': 0.0}

In [13]:
params = pp.loadflow.Parameters(distributed_slack=False)
sa = pp.sensitivity.create_dc_analysis()
sa.set_zones([zone_fr, zone_it])

In [14]:
sa.add_branch_flow_factor_matrix(branches_ids=['S_SE_1', 'S_SE_2'], variables_ids=['FR', 'IT'])

In [15]:
ptdf_results = sa.run(n6b, params)
m1 = ptdf_results.get_branch_flows_sensitivity_matrix()

In [16]:
m1

,S_SE_1,S_SE_2
FR,0.2,0.2
IT,-0.1,-0.1


1 MW active power transfer from FR zone to IT zone will be responsible of a variation of 0.3 MW on the border line S_SE_1.

Let’s obtain that directly. After a sensitivity analysis where we should set the zones, we are able to ask for a FR zone to slack sensitivity, a FR to IT zone to zone sensitivity, a IT to FR zone to zone sensitivity and a IT zone to slack sensitivity, on all the border lines ‘S_SE_1’, ‘S_SE_2’, ‘SE_NE_1’ and ‘SE_NE_2’.

In [17]:
sa = pp.sensitivity.create_dc_analysis()
sa.set_zones([zone_fr, zone_it])
sa.add_branch_flow_factor_matrix(branches_ids=['S_SE_1', 'S_SE_2', 'SE_NE_1', 'SE_NE_2'], variables_ids=['FR', ('FR', 'IT'), ('IT', 'FR'), 'IT'])
ptdf_result = sa.run(n6b, params)

In [18]:
m2 = ptdf_result.get_branch_flows_sensitivity_matrix()
m2

,S_SE_1,S_SE_2,SE_NE_1,SE_NE_2
FR,0.2,0.2,0.2,0.2
FR -> IT,0.3,0.3,-0.2,-0.2
IT -> FR,-0.3,-0.3,0.2,0.2
IT,-0.1,-0.1,0.4,0.4


We can see that:
- an increase of 1 MW on the FR zone net position leads to an increase of 0.2 MW on the flow of all the border lines
- moving the flow of 1 MW from FR to IT leads to an increase of 0.3 MW on the flow of lines S_SE_1 and S_SE_2 and a decrease of 0.2 MW on the flow of lines SE_NE_1 and SE_NE_2
- moving the flow of 1 MW from IT to FR leads to a decrease of 0.3 MW on the flow of lines S_SE_1 and S_SE_2 and an increase of 0.2 MW on the flow of lines SE_NE_1 and SE_NE_2
- an increase of 1 MW on the IT zone net position leads to a decrease of 0.1 MW on the flow of lines S_SE_1 and S_SE_2 and an increase of 0.4 MW on the flow of lines SE_NE_1 and SE_NE_2